In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
with open('/content/drive/My Drive/fr-en.en', "r") as f:
    english_fr_en = f.read()      

In [0]:
with open('/content/drive/My Drive/de-en.en', "r") as f:
    english_de_en = f.read()  
    

In [0]:
with open('/content/drive/My Drive/de-en.de', "r") as f:
    german_de_en = f.read()  

In [0]:
with open('/content/drive/My Drive/fr-en.fr', "r") as f:
    french_fr_en = f.read()  

In [0]:
from string import punctuation

In [0]:
import string
import nltk
import copy
english_fr_en_sentences = []
french_fr_en_sentences = []
english_de_en_sentences = []
german_de_en_sentences = []


def Text_Preprocess(text, language):
    lower_case_text = text.lower()
    for sentence in lower_case_text.split('\n'):
        out = "".join(l for l in sentence if l not in string.punctuation)
        if language == 'english_fr_en':
            english_fr_en_sentences.append(out)
        elif language == 'french_fr_en':
            french_fr_en_sentences.append(out)
        elif language == 'english_de_en':
            english_de_en_sentences.append(out)
        elif language == 'german_de_en':
            german_de_en_sentences.append(out)

Text_Preprocess(english_fr_en, 'english_fr_en')
Text_Preprocess(french_fr_en, 'french_fr_en')
Text_Preprocess(english_de_en, 'english_de_en')
Text_Preprocess(german_de_en, 'german_de_en')

temp = []
for sentence in english_fr_en_sentences:
    words = sentence.split()
    for word in words:
        temp.append(word)
        
unique_english_fr_en_vocab = set(temp)

temp = []
for sentence in french_fr_en_sentences:
    words = sentence.split()
    for word in words:
        temp.append(word)
        
unique_french_fr_en_vocab = set(temp)

temp = []
for sentence in english_de_en_sentences:
    words = sentence.split()
    for word in words:
        temp.append(word)
        
unique_english_de_en_vocab = set(temp)

temp = []
for sentence in german_de_en_sentences:
    words = sentence.split()
    for word in words:
        temp.append(word)
        
unique_german_de_en_vocab = set(temp)


In [0]:
english_fr_Translation_probabiliy = {}
french_fr_Translation_probabiliy = {}

for french_word in unique_french_fr_en_vocab:
     french_fr_Translation_probabiliy[french_word] = (1/len(unique_english_fr_en_vocab))

for eng_word in unique_english_fr_en_vocab:
    english_fr_Translation_probabiliy[eng_word] = copy.deepcopy(french_fr_Translation_probabiliy)

In [0]:
english_de_Translation_probabiliy = {}
german_de_Translation_probabiliy = {}

for german_word in unique_german_de_en_vocab:
     german_de_Translation_probabiliy[german_word] = (1/len(unique_english_de_en_vocab))

for eng_word in unique_english_de_en_vocab:
    english_de_Translation_probabiliy[eng_word] = copy.deepcopy(german_de_Translation_probabiliy)

In [0]:
import copy
def init_parameters(foreign_language):
    if foreign_language == 'german':
        english_vocab = unique_english_de_en_vocab
        foreign_language_vocab = unique_german_de_en_vocab
        probability_english = english_de_Translation_probabiliy
        english_sentences = english_de_en_sentences
        foreign_sentences = german_de_en_sentences
    if foreign_language == 'french':
        english_vocab = unique_english_fr_en_vocab
        foreign_language_vocab = unique_french_fr_en_vocab
        probability_english = english_fr_Translation_probabiliy
        english_sentences = english_fr_en_sentences
        foreign_sentences = french_fr_en_sentences
        
    clone_translation_probability = copy.deepcopy(probability_english)

    return [english_vocab, foreign_language_vocab, probability_english,  english_sentences,
            foreign_sentences, 0.002, 100, clone_translation_probability]

def init_total_f(foreign_language_vocab):
    total_f = {}
    for each_vocab in foreign_language_vocab:
        total_f[each_vocab] = 0
    return total_f

def counts_vocab_english(english_vocab, total_f):
    count = {}
    for each_english_vocab in english_vocab:
        count[each_english_vocab] = copy.deepcopy(total_f)
    return count

def construct_sum_array(sentence_words_e, sentence_words_foreign, clone_translation_probability):
    sum_words = {}
    for english_word in sentence_words_e:
            sum_words[english_word] = 0
            for foreign_word in sentence_words_foreign:
                    sum_words[english_word] += clone_translation_probability[english_word][foreign_word]
    return sum_words

def construct_total_f_array(sentence_words_e, sentence_words_foreign, clone_translation_probability,  count, total_of_f, sum_words):
    for english_word in sentence_words_e:
        for foriegn_word in sentence_words_foreign:
                count[english_word][foriegn_word] += clone_translation_probability[english_word][foriegn_word]/sum_words[english_word]                                       
                total_of_f[foriegn_word] += clone_translation_probability[english_word][foriegn_word]/sum_words[english_word]

def update_translation_probabilites(foreign_language_vocab, english_vocab, clone_translation_probability, count, total_of_f):
    for foreign_word in foreign_language_vocab:
         for english_word in english_vocab:
                 clone_translation_probability[english_word][foreign_word] = count[english_word][foreign_word]/total_of_f[foreign_word]

def sum_of_differenceProbabilites(english_vocab, foreign_language_vocab, clone_translation_probability, precounter_probabilities):
        sum_prob = 0
        for(english_word, foreign_word) in zip(english_vocab, foreign_language_vocab):
                    difference_in_prob = (clone_translation_probability[english_word][foreign_word] - precounter_probabilities[english_word][foreign_word])**2
                    sum_prob += difference_in_prob
        sum_prob = sum_prob**0.5   
        return sum_prob

def calculate_word_translation_probabilites(foreign_language):
    english_vocab = None
    foreign_language_vocab = None
    probability_english = None
    english_senteces = None
    foreign_sentences = None
    
    result = init_parameters(foreign_language)
    english_vocab = result[0]
    foreign_language_vocab = result[1]
    probability_english = result[2]
    english_sentences = result[3]
    foreign_sentences = result[4]
    threshold_limit_convergence = result[5]
    sum_prob = result[6]
    clone_translation_probability = result[7]
    iterations = 0
    
    while(sum_prob> threshold_limit_convergence):
            precounter_probabilities = copy.deepcopy(clone_translation_probability)
            total_of_f = init_total_f(foreign_language_vocab)
            
            counts = counts_vocab_english(english_vocab, total_of_f)
                
            for e_sentence, foreign_sentence in zip(english_sentences, foreign_sentences):
                sentence_words_e = e_sentence.split()
                sentence_words_foreign = foreign_sentence.split()
                sum_words = construct_sum_array(sentence_words_e, sentence_words_foreign, clone_translation_probability)
                construct_total_f_array(sentence_words_e, sentence_words_foreign, clone_translation_probability,  counts, total_of_f, sum_words)
            update_translation_probabilites(foreign_language_vocab, english_vocab, clone_translation_probability, counts, total_of_f)
            
    
            sum_prob = sum_of_differenceProbabilites(english_vocab, foreign_language_vocab, clone_translation_probability, precounter_probabilities)
            iterations = iterations + 1
            print("Iteration - {} - convergence_values - {}".format(iterations , sum_prob) )
    return clone_translation_probability

       


In [11]:
german_word_trans_prob = calculate_word_translation_probabilites('german')

Iteration - 1 - convergence_values - 0.1919498333132162
Iteration - 2 - convergence_values - 0.09230888667867092
Iteration - 3 - convergence_values - 0.10353983761769721
Iteration - 4 - convergence_values - 0.11826370444604828
Iteration - 5 - convergence_values - 0.12430896635456651
Iteration - 6 - convergence_values - 0.11546510819430872
Iteration - 7 - convergence_values - 0.09866295587398063
Iteration - 8 - convergence_values - 0.08036856286867924
Iteration - 9 - convergence_values - 0.06388475593045427
Iteration - 10 - convergence_values - 0.05029295406215737
Iteration - 11 - convergence_values - 0.03956160794865705
Iteration - 12 - convergence_values - 0.031247015000032418
Iteration - 13 - convergence_values - 0.024840345391393726
Iteration - 14 - convergence_values - 0.019897896541331988
Iteration - 15 - convergence_values - 0.01606927200834296
Iteration - 16 - convergence_values - 0.013088593251282974
Iteration - 17 - convergence_values - 0.010756677849480593
Iteration - 18 - co

In [12]:
for ew,german_data in german_word_trans_prob.items():
  for fw,pb in german_data.items():   
    if pb > 0.2:
      print("{}           {}          {}".format(ew, fw, pb))

accept           akzeptiere          0.5613114960464217
accept           annehmen          0.5344497905892809
accept           akzeptieren          0.7461523121675058
obligation           abkehr          0.3061301877627115
obligation           anzeigepflicht          0.3061301877627115
obligation           auferlegt          0.2627839095050137
obligation           moralische          0.2627839095050137
other           öffentlicher          0.43453327509583833
other           gewerbe          0.21029108449238754
other           fest          0.3397005348201229
other           weiteren          0.3515329769269629
other           andere          0.3106336849215157
other           verarbeitenden          0.21029108449238754
other           gern          0.4086705572269485
mandate           erfüllung          0.2189991758793539
mandate           mandats          0.2189991758793539
particular           ebenso          0.21383422736456578
will           überzeugt          0.49206506952065227


In [0]:
def print_german_alignment():
    
  for sentence in german_de_en_sentences:
    words = sentence.split()
    result_words = []
    
    for word in words:
      max_multiply = -70
      for trans_word in german_word_trans_prob.keys():
        if german_word_trans_prob[trans_word][word] > max_multiply:
            max_multiply = german_word_trans_prob[trans_word][word]
            max_word = trans_word
          
      result_words.append(max_word)
    print(result_words)
    print(sentence)


In [17]:
print_german_alignment()

['resumption', 'the', 'you']
wiederaufnahme der sitzungsperiode
['i', 'adjourned', 'the', 'on', 'friday', 'the', '17', 'december', 'adjourned', 'you', 'the', 'european', 'parliament', 'for', 'adjourned', 'like', 'you', 'once', 'everything', 'in', 'the', 'adjourned', 'and', 'hope', 'that', 'you', 'adjourned', 'adjourned', 'had']
ich erkläre die am freitag  dem 17 dezember unterbrochene sitzungsperiode des europäischen parlaments für wiederaufgenommen  wünsche ihnen nochmals alles gute zum jahreswechsel und hoffe  daß sie schöne ferien hatten 
['as', 'you', 'truly', 'were', 'is', 'the', 'truly', 'truly', 'not', 'truly', 'the', 'are', 'citizens', 'have', 'our', 'member', 'suffered', 'the', 'truly', 'disasters', 'i']
wie sie feststellen konnten  ist der gefürchtete  milleniumbug  nicht eingetreten  doch sind bürger einiger unserer mitgliedstaaten opfer von schrecklichen naturkatastrophen geworden 
['in', 'parliament', 'have', 'the', 'request', 'to', 'a', 'debate', 'in', 'few', 'this', 'you